In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [ ]:
analize = Analizer(0.1)
analize.Analize()

#analize.clean_folder(subfolder="dataset", extension="pkl")
#analize.clean_folder(subfolder="results", extension="xlsx")
#analize.clean_folder(subfolder="results", extension="txt")
#analize.clean_folder(subfolder="models", extension="keras", remove_last=False)


/tmp/ipykernel_3236060/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_3_0_1,0.982780,-0.120462,0.982788,0.987006,2.431817,165.694001,6.563919,2.598583,0.428674,1.559428,0.926529,1.583988,96.222723,202.767699,"Hidden Size=[12], regularizer=0.02, learning_r..."
1,model_3_17_7,0.999995,-0.122358,0.988511,0.999866,0.000750,165.974401,4.381486,0.001436,0.010222,0.027392,0.999977,0.027824,112.390001,218.934978,"Hidden Size=[12], regularizer=0.02, learning_r..."
2,model_3_17_3,0.999994,-0.125408,0.989832,0.999855,0.000896,166.425297,3.877554,0.001551,0.010701,0.029941,0.999973,0.030412,112.034157,218.579133,"Hidden Size=[12], regularizer=0.02, learning_r..."
3,model_3_10_4,0.999977,-0.129122,0.997675,0.999908,0.003257,166.974565,0.886668,0.014074,0.011531,0.057072,0.999902,0.057971,109.453747,215.998724,"Hidden Size=[12], regularizer=0.02, learning_r..."
4,model_3_10_3,0.999980,-0.130165,0.997856,0.999925,0.002836,167.128805,0.817792,0.011479,0.011333,0.053256,0.999914,0.054095,109.730561,216.275537,"Hidden Size=[12], regularizer=0.02, learning_r..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,model_1_25_6,0.999914,-3.456394,0.999254,0.999615,0.012165,659.011695,0.284386,0.058551,0.011019,0.110295,0.999632,0.112032,106.818376,213.363352,"Hidden Size=[12], regularizer=0.2, learning_ra..."
482,model_1_24_9,0.999914,-3.456394,0.999254,0.999523,0.012165,659.011695,0.284386,0.008115,0.011019,0.110295,0.999632,0.112032,106.818376,213.363352,"Hidden Size=[12], regularizer=0.2, learning_ra..."
483,model_1_26_2,0.999914,-3.456394,0.999254,0.999993,0.012165,659.011695,0.284386,0.001060,0.011019,0.110295,0.999632,0.112032,106.818376,213.363352,"Hidden Size=[12], regularizer=0.2, learning_ra..."
484,model_1_26_9,0.999914,-3.456394,0.999254,0.999993,0.012165,659.011695,0.284386,0.001060,0.011019,0.110295,0.999632,0.112032,106.818376,213.363352,"Hidden Size=[12], regularizer=0.2, learning_ra..."
